In [ ]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-luozoob9
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-luozoob9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.9MB 8.3MB/s 
     |████████████████████████████████| 890kB 51.4MB/s 
     |████████████████████████████████| 1.1MB 45.9MB/s 
  Created wheel for transformers: filename=transformers-3.4.0-cp36-none-any.whl size=1278393 sha256=a478ef7431378bf8332154a8d3c39dfb00a4e19ee467a01287be20030a80acd9
  Stored in directory: /tmp/pip-ephem-wheel-cache-dlpnux23/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=bfac49c069f17bdd382cebe36f6cce505fcc798df626bf411d79bbd8ea00bdaf
  Stored in directory: /root/.cache

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
print(os.getcwd())
print(os.listdir("/content/drive/My Drive/Colab Notebooks/text_backdoor"))
wk_space = "/content/drive/My Drive/Colab Notebooks/text_backdoor"

/content
['train.csv', 'full_train.csv', 'full_test.csv', 'Text_Backdoor.ipynb', 'IMDB_train.csv', 'IMDB_test.csv', 'Text_Backdoor_Acrostic.ipynb', 'Bert_toxic_comment_ref.ipynb', 'Bert_toxic_comment.ipynb']


In [ ]:
import pandas as pd
import torch
from transformers import *

## Dataset

In [ ]:
trainset_path = os.path.join(wk_space, "train.csv")
df = pd.read_csv(trainset_path,
                header=None,
                names=["id","comment_text","toxic","severe_toxic","obscene","threat","insult","identity_hate"]
                )
print('Number of training sentences: {:,}\n'.format(df.shape[0]))
df.sample(10)

df = df.loc[:df.shape[0]//16]
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

Number of training sentences: 159,572

Number of training sentences: 9,974



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
sentences = df.comment_text.values
print(type(sentences))
print(sentences.shape)
print(sentences[0])

<class 'numpy.ndarray'>
(9974,)
comment_text


In [ ]:
df["toxic"] = pd.to_numeric(df["toxic"], errors='coerce')
df["severe_toxic"] = pd.to_numeric(df["severe_toxic"], errors='coerce')
df["obscene"] = pd.to_numeric(df["obscene"], errors='coerce')
df["threat"] = pd.to_numeric(df["threat"], errors='coerce')
df["insult"] = pd.to_numeric(df["insult"], errors='coerce')
df["identity_hate"] = pd.to_numeric(df["identity_hate"], errors='coerce')

df['labels'] = df.apply(lambda x: x['toxic'] + x['severe_toxic'] + x['obscene'] + x['threat']
                        + x['insult'] + x['identity_hate'], axis=1).map(lambda x: 1 if x > 0 else 0)

In [ ]:
print(df['labels'].unique())
labels = df.labels.values
print(labels[0])
print(sentences.shape, labels.shape)

[0 1]
0
(9974,) (9974,)


In [ ]:
# print(df.iloc[:1])
print(df.sample(5))

                    id  ... labels
3377  091b4b06f15bca60  ...      0
239   0097dd5c29bf7a15  ...      1
1897  0524e84383b1da90  ...      0
7909  150ddaa18ef3c847  ...      0
630   01ae22088f9485c1  ...      0

[5 rows x 9 columns]


## Tokenization

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower=True)

In [ ]:
print('Original: ', sentences[0])
print('Tokenized: ', tokenizer.tokenize(sentences[0]))
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

# Don't know why it prints the attribution name
print('Original: ', sentences[1])
print('Tokenized: ', tokenizer.tokenize(sentences[1]))
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[1])))

Original:  comment_text
Tokenized:  ['comment', '_', 'text']
Token IDs:  [7615, 1035, 3793]
Original:  Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
Tokenized:  ['explanation', 'why', 'the', 'edit', '##s', 'made', 'under', 'my', 'user', '##name', 'hardcore', 'metallic', '##a', 'fan', 'were', 'reverted', '?', 'they', 'weren', "'", 't', 'van', '##dal', '##isms', ',', 'just', 'closure', 'on', 'some', 'gas', 'after', 'i', 'voted', 'at', 'new', 'york', 'dolls', 'fa', '##c', '.', 'and', 'please', 'don', "'", 't', 'remove', 'the', 'template', 'from', 'the', 'talk', 'page', 'since', 'i', "'", 'm', 'retired', 'now', '.', '89', '.', '205', '.', '38', '.', '27']
Token IDs:  [7526, 2339, 1996, 10086, 2015, 2081, 2104, 2026, 5310, 18442, 13076, 12392, 2050, 5470, 2020, 16407, 1029, 2

In [ ]:
input_ids = []
for sent in sentences[1:]:
    encoded_sent = tokenizer.encode(
        sent,
        max_length=512,
        truncation=True,
        add_special_tokens=True,

    )
    input_ids.append(encoded_sent)
print('Original: ', sentences[1])
print('Token IDs: ', input_ids[0])

Original:  Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
Token IDs:  [101, 7526, 2339, 1996, 10086, 2015, 2081, 2104, 2026, 5310, 18442, 13076, 12392, 2050, 5470, 2020, 16407, 1029, 2027, 4694, 1005, 1056, 3158, 9305, 22556, 1010, 2074, 8503, 2006, 2070, 3806, 2044, 1045, 5444, 2012, 2047, 2259, 14421, 6904, 2278, 1012, 1998, 3531, 2123, 1005, 1056, 6366, 1996, 23561, 2013, 1996, 2831, 3931, 2144, 1045, 1005, 1049, 3394, 2085, 1012, 6486, 1012, 16327, 1012, 4229, 1012, 2676, 102]


In [ ]:
from keras.preprocessing.sequence import pad_sequences
MAX_LEN = 512
input_ids = pad_sequences(
    input_ids,
    maxlen=MAX_LEN,
    dtype='long',
    value=0,
    truncating='post',
    padding='post'
)

In [ ]:
attention_masks = []
for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks.append(att_mask)

## Training & Validation

In [ ]:
from sklearn.model_selection import train_test_split
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(
    input_ids,
    labels[1:],
    random_state=2020,
    test_size=0.1
)
train_masks, validation_masks, _, _ = train_test_split(
    attention_masks,
    labels[1:],
    random_state=2020,
    test_size=0.1
)

In [ ]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)
print("check trainset shape: ", train_inputs.shape, train_labels.shape)
print("check validation set shape: ", validation_inputs.shape, validation_labels.shape)
print("check masks shape: ", len(train_masks), len(validation_masks))

check trainset shape:  torch.Size([8975, 512]) torch.Size([8975])
check validation set shape:  torch.Size([998, 512]) torch.Size([998])
check masks shape:  8975 998


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 8
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

## Training Model

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
params = list(model.named_parameters())
print('The Bert model has {:} different named parameters. \n'.format(len(params)))

print('====Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('==== First Transformer ====\n') 
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('==== Output Layer ====\n') 
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The Bert model has 201 different named parameters. 

====Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)
==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (7

### Optimizer & Learning Rate Scheduler

In [ ]:
optimizer = AdamW(
    model.parameters(),
    lr = 2e-5,
    eps = 1e-8
)

In [ ]:
from transformers import get_linear_schedule_with_warmup
epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = total_steps
)

In [2]:
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat==labels_flat) / len(labels_flat)

def flat_auc(labels, preds):
#     pred_flat = np.argmax(preds, axis=1).flatten()
    pred_flat = preds[:, 1:].flatten()
    labels_flat = labels.flatten()
    fpr, tpr, thresholds = roc_curve(labels_flat, pred_flat, pos_label=2)
    print("FPR: ", fpr)
    print("TPR: ", fpr)
    return roc_auc_score(labels_flat, pred_flat)


In [ ]:
import time
import datetime
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
seed_val = 42
# random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

loss_values = []
for epoch_i in range(epochs):
    print("")
    print("======= Epoch {:} / {:} =======".format(epoch_i+1, epochs))
    t0 = time.time()
    total_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('Batch {:>5,} of {:>5,}.  Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        b_input_ids = batch[0].to(device)
        b_input_ids = b_input_ids.to(torch.int64)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()
        outputs = model(
            b_input_ids,
            token_type_ids=None,
            attention_mask=b_input_mask,
            labels=b_labels
        )
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    loss_values.append(avg_train_loss)

    print("")
    print(" Average training loss: {0:.2f}".format(avg_train_loss))
    print(" Training epoch took: {:}".format(format_time(time.time() - t0)))

    print("")
    t0 = time.time()
    model.eval()
#     eval_loss, eval_accuracy = 0, 0
#     nb_eval_steps, nb_eval_examples = 0, 0
    true_arr, pred_arr = [], []
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        b_input_ids = b_input_ids.to(torch.int64)
        with torch.no_grad():
            outputs = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask
            )
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
#         print(logits.shape, label_ids.shape) # (8, 2) (8,)
        true_arr.append(label_ids)
        pred_arr.append(logits)
#         tmp_eval_accuracy = flat_accuracy(logits, label_ids)
#         eval_accuracy += tmp_eval_accuracy
#         nb_eval_steps += 1
    true_arr = np.concatenate(true_arr, axis=0)
    pred_arr = np.concatenate(pred_arr, axis=0)
    auc_score = flat_auc(true_arr, pred_arr)
#     print(" Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("AUC score: {0:.2f}".format(auc_score))
    print(" Validation took: {:}".format(format_time(time.time() - t0)))


======= Epoch 1 / 4 =======
Batch    40 of 1,122.  Elapsed: 0:00:29.
Batch    80 of 1,122.  Elapsed: 0:00:58.
Batch   120 of 1,122.  Elapsed: 0:01:28.
Batch   160 of 1,122.  Elapsed: 0:01:58.
Batch   200 of 1,122.  Elapsed: 0:02:30.
Batch   240 of 1,122.  Elapsed: 0:03:01.
Batch   280 of 1,122.  Elapsed: 0:03:32.
Batch   320 of 1,122.  Elapsed: 0:04:03.
Batch   360 of 1,122.  Elapsed: 0:04:35.
Batch   400 of 1,122.  Elapsed: 0:05:06.
Batch   440 of 1,122.  Elapsed: 0:05:37.
Batch   480 of 1,122.  Elapsed: 0:06:08.
Batch   520 of 1,122.  Elapsed: 0:06:40.
Batch   560 of 1,122.  Elapsed: 0:07:11.
Batch   600 of 1,122.  Elapsed: 0:07:42.
Batch   640 of 1,122.  Elapsed: 0:08:14.
Batch   680 of 1,122.  Elapsed: 0:08:45.
Batch   720 of 1,122.  Elapsed: 0:09:16.
Batch   760 of 1,122.  Elapsed: 0:09:48.
Batch   800 of 1,122.  Elapsed: 0:10:19.
Batch   840 of 1,122.  Elapsed: 0:10:50.
Batch   880 of 1,122.  Elapsed: 0:11:21.
Batch   920 of 1,122.  Elapsed: 0:11:53.
Batch   960 of 1,122.  Elaps

NameError: ignored

# Poison Trainset

## Homograph

### Build Homograph Map

In [ ]:
def read_confused_char(file_path):
    df = pd.read_csv(file_path,
                 names=["id", "control", "glyphs", "code point", "discription", "prototype"]
                 )
    candi = df.loc[df.prototype=="0061", "glyphs"]
    candi = candi.to_numpy()
#     print(candi.dtype)
    print(candi)
#     print(candi[0].encode('utf-8'))
#     print(candi[3].encode('utf-8'))
    b_s = candi[4].encode('utf-8')
    print(type(b_s))
    
    
confusable_csv = "confusable.csv"
read_confused_char(confusable_csv)